In [2]:
from tkinter import *
import numpy as np
from scipy import stats
import sys

tk=Tk()
tk.title('Option Pricing')
tk.geometry('540x560')

def Black_Scholes():
       S = float(S_entry.get())  
       K = float(K_entry.get())  
       T = float(T_entry.get())  
       r = float(r_entry.get())   
       Sigma = float(Sigma_entry.get()) 
       #CallorPut = CallorPut_entry.get()  #買權或賣權
       
       d1 = (np.log(S/K) + (r+0.5*Sigma**2)*T)/(Sigma*np.sqrt(T))
       d2 = d1 - Sigma*np.sqrt(T)
    
       if    (var1.get() == 1) & (var2.get() == 0):
        #CallorPut == 'Call':
             result = S*stats.norm.cdf(d1) - K*np.exp(-r*T)*stats.norm.cdf(d2)
                
       elif  (var1.get() == 0) & (var2.get() == 1):
         #CallorPut == 'Put':
             result = K*np.exp(-r*T)*stats.norm.cdf(-d2) - S*stats.norm.cdf(-d1)
            
       else:
             result='Wrong!'
                
       result_label['text']=round(result,6)

def delta():
    S = float(S_entry.get())
    K = float(K_entry.get())
    T = float(T_entry.get())  
    r = float(r_entry.get())   
    Sigma = float(Sigma_entry.get()) 
    d1 = (np.log(S/K) + (r+0.5*Sigma**2)*T)/(Sigma*np.sqrt(T))
    if    (var1.get() == 1) & (var2.get() == 0):
             delta=stats.norm.cdf(d1)
    elif  (var1.get() == 0) & (var2.get() == 1):  
             delta=stats.norm.cdf(d1)-1
    else:
             delta='Wrong!'
    delta_label['text']=round(delta,6)       
    
def gamma():
    S = float(S_entry.get())
    K = float(K_entry.get())
    T = float(T_entry.get())  
    r = float(r_entry.get())   
    Sigma = float(Sigma_entry.get()) 
    d1 = (np.log(S/K) + (r+0.5*Sigma**2)*T)/(Sigma*np.sqrt(T))
    gamma=stats.norm.pdf(d1)/(S*Sigma*np.sqrt(T)) #gamma for call/put are the same
    gamma_label['text']=round(gamma,6) 

def theta():
    S = float(S_entry.get())
    K = float(K_entry.get())
    T = float(T_entry.get())  
    r = float(r_entry.get())   
    Sigma = float(Sigma_entry.get()) 
    d1 = (np.log(S/K) + (r+0.5*Sigma**2)*T)/(Sigma*np.sqrt(T))
    d2 = d1 - Sigma*np.sqrt(T)
    if    (var1.get() == 1) & (var2.get() == 0):
            theta=-(S*Sigma*stats.norm.pdf(d1))/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*stats.norm.cdf(d2)
    elif  (var1.get() == 0) & (var2.get() == 1):
            theta=-(S*Sigma*stats.norm.pdf(d1))/(2*np.sqrt(T)) + r*K*np.exp(-r*T)*stats.norm.cdf(-d2)
    else:
        theta='Wrong!'
    theta_label['text']=round(theta,6)    

def vega():
    S = float(S_entry.get())
    K = float(K_entry.get())
    T = float(T_entry.get())  
    r = float(r_entry.get())   
    Sigma = float(Sigma_entry.get())
    d1 = (np.log(S/K) + (r+0.5*Sigma**2)*T)/(Sigma*np.sqrt(T))
    vega=S*np.sqrt(T)*stats.norm.pdf(d1)  #vega for call/put are the same
    vega_label['text']=round(vega,6)

def rho():
    S = float(S_entry.get())
    K = float(K_entry.get())
    T = float(T_entry.get())  
    r = float(r_entry.get())   
    Sigma = float(Sigma_entry.get())
    d1 = (np.log(S/K) + (r+0.5*Sigma**2)*T)/(Sigma*np.sqrt(T))
    d2 = d1 - Sigma*np.sqrt(T)
    if    (var1.get() == 1) & (var2.get() == 0):
        rho=K*T*np.exp(-r*T)*stats.norm.cdf(d2)
    elif  (var1.get() == 0) & (var2.get() == 1):
        rho=-K*T*np.exp(-r*T)*stats.norm.cdf(-d2)
    else:    
        rho='Wrong!'
    rho_label['text']=round(rho,6)    

def binomial():
   
    S = float(S_entry.get())
    K = float(K_entry.get())
    T = float(T_entry.get())  
    r = float(r_entry.get())   
    Sigma = float(Sigma_entry.get())
    M=500
    dt=T/M #split T by M
    df=np.exp(-r*dt) #discount factor
    u=np.exp(Sigma*np.sqrt(dt)) #up movement
    d=1/u
    p=(np.exp(r*dt)-d)/(u-d) #risk-neutral probability

    S1=np.zeros((M+1,M+1),dtype=np.float64)
    S1[0,0]=S
    
    for j in range(1,M+1):
        for i in range(j+1):
            S1[i,j]=S1[0,0]*(u**j)*(d**(i*2))  #每一個節點的股價
            
    value=np.zeros((M+1,M+1),dtype=np.float64)
    for j in range(0,M+1):
        for i in range(j+1):
            if    (var1.get() == 1) & (var2.get() == 0):
                    value[i,j]=max(S1[i,j]-K,0) #每一個節點的call payoff
            elif  (var1.get() == 0) & (var2.get() == 1):
                    value[i,j]=max(K-S1[i,j],0)  #每一個節點的put payoff
            
    
    price=np.zeros((M+1,M+1),dtype=np.float64)
    price[:,M]=value[:,M]
    for j in range(M-1,-1,-1):
        for i in range(j+1):
                price[i,j]=(p*price[i,j+1]+(1-p)*price[i+1,j+1])*df #每一個節點的call/put price
                result1=price[0,0]
                
    result1_label['text']=round(result1,6)

def clear_text():
    S_entry.delete(0,END)
    K_entry.delete(0,END)
    r_entry.delete(0,END)
    Sigma_entry.delete(0,END)
    T_entry.delete(0,END)
    var1.set(0)
    var2.set(0)
    
        
Label(tk,text="Underlying Price").grid(row=0,sticky=W)
S_entry=Entry(tk)
S_entry.grid(row=0,column=1)

Label(tk,text="Strike Price").grid(row=1,sticky=W)
K_entry=Entry(tk)
K_entry.grid(row=1,column=1)

Label(tk,text="Interest Rate").grid(row=2,sticky=W)
r_entry=Entry(tk)
r_entry.grid(row=2,column=1)

Label(tk,text="Sigma").grid(row=3,sticky=W)
Sigma_entry=Entry(tk)
Sigma_entry.grid(row=3,column=1)

Label(tk,text="Time to Maturity").grid(row=4,sticky=W)
T_entry=Entry(tk)
T_entry.grid(row=4,column=1)

Label(tk,text="Call or Put").grid(row=5,sticky=W)
#CallorPut_entry=Entry(tk)
#CallorPut_entry.grid(row=5,column=1)

Label(tk,text="").grid(row=7,sticky=W)
Label(tk,text="～～～～～*****RESULT*****～～～～～").grid(row=8,sticky=W)
Label(tk,text="").grid(row=9,sticky=W)

result_label=Label(tk,text="price")
result_label.grid(row=10,column=1)
result1_label=Label(tk,text="binomial price")
result1_label.grid(row=16,column=1)
delta_label=Label(tk,text="delta")
delta_label.grid(row=11,column=1)
gamma_label=Label(tk,text="gamma")
gamma_label.grid(row=12,column=1)
theta_label=Label(tk,text="theta")
theta_label.grid(row=13,column=1)
vega_label=Label(tk,text="vega")
vega_label.grid(row=14,column=1)
rho_label=Label(tk,text="rho")
rho_label.grid(row=15,column=1)

var1=IntVar(tk)
var2=IntVar(tk)
c1=Checkbutton(tk,text="Call",variable=var1, onvalue=1, offvalue=0)
c1.grid(row=5,column=1,sticky=W+E,pady=2)
c2=Checkbutton(tk,text="Put",variable=var2, onvalue=1, offvalue=0)
c2.grid(row=6,column=1,sticky=W+E,pady=4)

button1=Button(tk,text="Option Price",command=Black_Scholes)
button1.grid(row=10,column=0)
button2=Button(tk,text="      Delta       ",command=delta)
button2.grid(row=11,column=0)
button3=Button(tk,text="     Gamma    ",command=gamma)
button3.grid(row=12,column=0) 
button4=Button(tk,text="      Theta      ",command=theta)
button4.grid(row=13,column=0)
button5=Button(tk,text="      Vega       ",command=vega)
button5.grid(row=14,column=0)
button6=Button(tk,text="       Rho        ",command=rho)
button6.grid(row=15,column=0)
button7=Button(tk,text="Binomial Price",command=binomial)
button7.grid(row=16,column=0)

Label(tk,text="").grid(row=17,sticky=W)
button8=Button(tk,text="Clear",command=clear_text)
button8.grid(row=19,column=6,sticky=W+E,pady=2)
button8=Button(tk,text="Quit",command=tk.destroy)
button8.grid(row=20,column=6,sticky=W+E,pady=4)

button10=Button(tk,text="    All result    ",command=lambda:[Black_Scholes(),delta(),gamma(),theta(),vega(),rho(),binomial()])
button10.grid(row=17,column=0)
tk.mainloop()